<a href="https://colab.research.google.com/github/anushka-code/Code-Smell-Classification/blob/main/Code_Smell_Classification_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code Smell Detection (Smelly/Non Smelly) using Structural & Semantic Features 

#Code Smells Targetted: 


1.   Long Parameters List
2.   Switch Statements







###Importing Libraries

In [1]:
#!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Functional, Model, Sequential
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, concatenate
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
import imblearn
from imblearn.over_sampling import SMOTE
import nltk 
from nltk.tokenize import RegexpTokenizer
import re
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

# import transformers
# from transformers import DistilBertTokenizer
# from transformers import TFDistilBertForSequenceClassification
# from transformers import TFDistilBertModel

###Mounting Google Drive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Dataset Loader

In [4]:
def DataLoader(link, name_of_file):
  id = link.split("/")[-2]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(name_of_file)
  dataframe = pd.read_csv(name_of_file)
  return dataframe

link1 = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
link2 = 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
link3 = 'https://drive.google.com/file/d/1xeMRB6ACyjep6VF_rDRq8cXkA58ZlEZs/view?usp=sharing'
link4 = 'https://drive.google.com/file/d/1OxxMX7-kc7mQUvb2DSJR_fbtvZ5bB1Ef/view?usp=sharing'
link5 = 'https://drive.google.com/file/d/1mRIgoXhLwCdao7f9yD2QTSi7JNZNq9zJ/view?usp=sharing'
link6 = 'https://drive.google.com/file/d/1rLkJAwHkBAAkHMp2L1Y1AzuniIZdyB7x/view?usp=sharing'
name1 = 'long_parameters_list_structural.csv'
name2 = 'switch_statements_structural.csv'
name3 = 'long_parameters_list_semantic.csv'
name4 = 'switch_statements_semantic.csv'
name5 = 'multilabel_classification_dataset.csv'
name6 = 'semantic_final.csv'

df_lp = DataLoader(link1, name1)
df_ss = DataLoader(link2, name2)
df_lpm = DataLoader(link3, name3)
df_ssm = DataLoader(link4, name4)
df_multilabel = DataLoader(link5, name5)
df_semantic = DataLoader(link6,name6)

Long Parameters List Structural Dataset

In [5]:
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,...,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,...,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,...,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Structural Dataset



In [6]:
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_switch_statements
0,1,2,0,0,2,1,5,1,0,1,...,3,439,53,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,2,6,3,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,2,0,0,0,0,1,3,1,0,2,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,0,0,0,0,2,3,1,0,0,...,0,1212,120,43,849,78,5788,4778,65687,False
4,0,0,0,0,0,1,3,1,0,0,...,0,516,29,43,849,78,5788,4778,65687,False


Long Parameters List Semantic Dataset

In [7]:
df_lpm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.common.plugins,ReminderHookBase,shutdownMapHook(),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapMapModel.StringReaderCreator,createReader(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapController,newModel(ModeController),False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,MindMapNode,getColor(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,ControllerAdapter.ControllerPopupMenuListener,popupMenuCanceled(PopupMenuEvent),False


Switch Statements Semantic Dataset

In [8]:
df_ssm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.actions.xml,AbstractXmlAction,actionPerformed(ActionEvent),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,EncryptedMindMapNode,childrenUnfolded(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,freemind.modes.mindmapmode.MindMapController$1,"apply(MindMapMapModel,MindMapNodeModel)",False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.view.mindmapview.attributeview,freemind.view.mindmapview.attributeview.Attrib...,run(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.listeners,MindMapNodeMotionListener,stopDrag(),False


In [9]:
# df_multilabel.head(5)

In [10]:
# df_multilabel.shape

# Data Pre-Processing of Structural Dataset


In [11]:
def PrePro(last_column, dataframe):
  dataframe.rename(columns = {last_column :'is_code_smell'}, inplace = True) #rename column
  dataframe['is_code_smell'] = dataframe["is_code_smell"].astype(int) #change boolean labels to int labels
  Y_part = dataframe.iloc[:,-1:]
  X_part = dataframe.iloc[:,:56]
  X_part = X_part.replace(to_replace =["?"], value = np.nan) #replace non existing values with null
  X_part = X_part.astype(float) #change datatype of features of X as float
  return X_part,Y_part

X_lp, Y_lp = PrePro('is_long_parameters_list',df_lp)
X_ss, Y_ss = PrePro('is_switch_statements',df_ss)

# X_m, Y_m = PrePro(df_multilabel)

In [12]:
def MeanforNaN(dataframe):   #function to fill null spaces with column mean 
  column_means = dataframe.mean()
  dataframe = dataframe.fillna(column_means)
  return dataframe

X_lp = MeanforNaN(X_lp)
X_ss = MeanforNaN(X_ss)

In [13]:
def ConCat(df1,df2): #concatenate code smell datasets
  code_smells = [df1,df2]
  joint = pd.concat(code_smells)
  return joint

X_train = ConCat(X_lp,X_ss)
Y_train = ConCat(Y_lp,Y_ss)

###Data Normalization

In [14]:
def Normalize(dataframe): #apply MinMax normalisation to fit the values between 0 to 1
  scaler = MinMaxScaler()
  model = scaler.fit(dataframe)
  scaled_data = model.transform(dataframe)
  return scaled_data

# X_m_sample = Normalize(X_m)
X_sample = Normalize(X_train)

###Synthetic Minority Oversampling Technique (SMOTE) Algo for Imbalanced Datasets


In [15]:
Y_train.value_counts() #1/3rd are smelly, 2/3rd are non smelly 

is_code_smell
0                573
1                267
dtype: int64

In [16]:
Y_sample = Y_train.to_numpy(dtype='int64', copy='True')

In [17]:
def Oversample(X_data,Y_data): # Using Smote obtain a 50-50 balanced dataset 

  sm = SMOTE(random_state = 2)
  X_train_res, Y_train_res = sm.fit_resample(X_data, Y_data.ravel())
  return X_train_res, Y_train_res

X_new, Y_new = Oversample(X_sample,Y_sample)

In [18]:
# X_train_final = X_new.reshape((1146,56,1)) # hence makes 573 smelly and 573 non smelly samples

In [19]:
X_struct = pd.DataFrame(X_new)

In [20]:
Y_train_final = Y_new.reshape(1146,)

#Word Embeddings using Gensim Word2Vec

In [21]:
# def WordEmbedding(tokenizedwords): #creates a dictionary with unique vocab words and corresponding vectors 
#   model = Word2Vec(tokenizedwords, min_count=1, sg=1, size=50) #skipgram model 
#   model.train(tokenizedwords, total_examples=model.corpus_count, epochs=10)
#   keys = list(model.wv.vocab.keys())
#   values = list(model.wv.vectors)
#   wdict = {}
#   for i in range(840):
#     wdict[keys[i]] = values[i]
#   return wdict

# word_vec = WordEmbedding(word_groups) #this dictionary contains the key as word and value as the vector

In [22]:
# def VectorRep(groupofwords,dictionary): #gives a list of list which contains vectors for each word 
#   rep=[]
#   for i in groupofwords:
#     l=[]
#     for j in i:
#       l.append(dictionary.get(j))
#     rep.append(l)
#   return rep

# vector_rep = VectorRep(word_groups, word_vec)

In [23]:
# def AvgLength(vectorrepresentation):
#   sum = 0
#   for i in range(840):
#     sum += len(vectorrepresentation[i])
#   avg = sum/840
#   return avg

# AvgLength(vector_rep)

In [24]:
# def FindMaxLength(lst):
#     maxList = max((x) for x in lst)
#     maxLength = max(len(x) for x in lst )
  
#     return maxList, maxLength
      
# print(FindMaxLength(word_groups))

In [25]:
# x = np.zeros(50)
# print(x)
# type(x)
# m = x.tolist()
# type(m)

In [26]:
# tensor1 = tf.zeros(4,4)
# print(tensor1)

In [27]:
# tensor = tf.convert_to_tensor(vector_rep[0]) 
# print(tensor.shape)

In [28]:
# type(vector_rep)

#Text Pre Processing

In [29]:
# df_sem = ConCat(df_lpm,df_ssm) #join long parameters list and switch statements dataset

In [30]:
def PrePro2(dataframe):
  X_part = dataframe.iloc[:,:-1]  
  return X_part

X_sem= PrePro2(df_semantic) #separate as X and Y columns

In [31]:
def ToList(dataframe, string): #converts pandas.core.series.Series to list of lists
  new_column = dataframe[string].tolist()
  return list(map(lambda x:[x], new_column))

class_words = ToList(X_sem, 'class')
method_words = ToList(X_sem, 'method')

###Tokenization

In [32]:
def TokenizeGroups(text): #tokenizes as per CamelCase RegEx and converts to lowercase
  tokenizer = RegexpTokenizer('[a-zA-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))') # RegEx pattern for CamelCase 
  useful_text = tokenizer.tokenize(text) 
  useful_text = [x.lower() for x in useful_text]
  return useful_text

In [33]:
def LoopOver(list): #loops over all the samples to tokenize all the strings in each word group
  for i in range(0,1146):
    list[i] = TokenizeGroups(list[i][0])
    i=i+1
  return list

class_words_sem = LoopOver(class_words)
method_words_sem = LoopOver(method_words)

In [34]:
def WordList(list1,list2): #concatenate the words of each project, package, class and method
  final_list = []
  for i in range(0,1146):
    x = list1[i] + list2[i]
    final_list.append(x)
  res = [' '.join(ele) for ele in final_list]
  df = pd.DataFrame(res)
  return df

word_groups = WordList(class_words_sem, method_words_sem)

In [35]:
word_groups.columns = [56]
word_groups

,56
0,reminder hook base shutdown map hook
1,mind map map model string reader creator creat...
2,mind map controller new model mode controller
3,mind map node get color
4,controller adapter controller popup menu liste...
...,...
1141,byte buffer output stream byte buffer output s...
1142,byte buffer output stream close array list int...
1143,traffic get traffic results string string stri...
1144,traffic get traffic results string int int int...


In [36]:
# def Sentences(group_of_words): #makes a list of lists containing sentences for each single sample 
#   lst = []
#   for i in range(0,1146):
#     appendlist = []
#     string = group_of_words[i]
#     appendstr = ' '.join(string)
#     appendlist.append(appendstr)
#     lst.append(appendlist)
#   return lst

# sentence_groups = Sentences(word_groups)

In [37]:
# token = Tokenizer()
# token.fit_on_texts(word_groups)
# vocab_size = len(token.word_index) + 1
# print(vocab_size)

In [38]:
# encoded_text = token.texts_to_sequences(word_groups)

In [39]:
# max_length = 176
# X = pad_sequences(encoded_text, maxlen = max_length, padding = 'pre')
# print(X)

In [40]:
# token.word_index.items()

In [41]:
# X.shape

### Random Forest Classifier

###Glove Vectors

In [42]:
# import gensim.downloader as api
# glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

In [43]:
# vector_size = 100 
# gensim_weight_matrix = np.zeros(( vocab_size ,vector_size)) 
# gensim_weight_matrix.shape

In [44]:
# for word, index in token.word_index.items():
#     if index < vocab_size: 
#         if word in glove_gensim.wv.vocab:
#             gensim_weight_matrix[index] = glove_gensim[word]
#         else:
#             gensim_weight_matrix[index] = np.zeros(100)

###Stratified K-Fold Cross Validation for Limited Data


In [45]:
# def CrossValidation(dataX,dataY): #split the dataset into train and test using k-5 fold cross validation
#   skf = StratifiedKFold(n_splits=5, random_state = None, shuffle=True) 

#   for train_index, test_index in skf.split(dataX , dataY):
#         X_crosstrain, X_crosstest = dataX[train_index], dataX[test_index] 
#         Y_crosstrain, Y_crosstest = dataY[train_index], dataY[test_index]
#         return X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest

# X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest = CrossValidation(X_train_final, Y_train_final)

### ML Model - Pipeline of Feature Union with RF Classifier & GridSearchCV

In [48]:
X_final = pd.concat([X_struct, word_groups], axis=1) # combining the structural and semantic data

In [49]:
X_ml_final = X_final.to_numpy() # converting df into numpy array

In [50]:
def CrossValidation(dataX,dataY): #split the dataset into train and test using k-5 fold cross validation
  skf = StratifiedKFold(n_splits=5, random_state = None, shuffle=True) 

  for train_index, test_index in skf.split(dataX , dataY):
        X_crosstrain, X_crosstest = dataX[train_index], dataX[test_index] 
        Y_crosstrain, Y_crosstest = dataY[train_index], dataY[test_index]
        return X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest

X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest = CrossValidation(X_ml_final, Y_train_final)

In [51]:
def get_numeric_data(x):
    return [record[:-2].astype(float) for record in x] #gets the numeric data which is all the columns except last column

def get_text_data(x):
    return [record[-1] for record in x] # gets the text data which is the last column of the dataset

In [52]:
transformer_numeric = FunctionTransformer(get_numeric_data) #transformer_numeric when transformed on some X will apply get_numeric_data on X
transformer_text = FunctionTransformer(get_text_data)

In [53]:
pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transformer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(analyzer='word'))
            ]))
         ])),
    ('clf', RandomForestClassifier())
])

In [108]:
param_grid = {'clf__n_estimators': np.linspace(1, 100, 42, dtype=int),
              'clf__min_samples_split': [2],
              'clf__min_samples_leaf': [1],
              'clf__max_features': [8],
              'clf__max_depth': [None],
              'clf__criterion': ['entropy'],
              'clf__bootstrap': [False]}

scoring = {'Accuracy': 'accuracy', 'F1': 'f1_macro'}
refit = 'F1'

In [109]:
rf_model = GridSearchCV(pipeline, param_grid=param_grid, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=1)

In [110]:
rf_model.fit(X_crosstrain, Y_crosstrain)
# rf_best = rf_model.best_estimator_
# print(rf_best)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('numeric_features',
                                                                        Pipeline(steps=[('selector',
                                                                                         FunctionTransformer(func=<function get_numeric_data at 0x7fb4f742e5f0>))])),
                                                                       ('text_features',
                                                                        Pipeline(steps=[('selector',
                                                                                         FunctionTransformer(func=<function get_text_data at 0x7fb4f742e7a0>)),
                                                                                        ('vec',
                                                                                         TfidfVectorizer())]))])),
                            

In [111]:
rf_model.score(X_crosstest, Y_crosstest)

0.9086110007379236

### Bi-LSTM

In [ ]:
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout

In [ ]:
EMBEDDING_DIM = 100
class_num = 1
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = EMBEDDING_DIM, input_length= X.shape[1],
                    weights = [gensim_weight_matrix],trainable = False))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',  metrics = ['accuracy'])
model.summary()

In [ ]:
# model.fit(X, Y_train_final , epochs = 50, batch_size = 100)

###distilBERT

In [ ]:
# def Sentences(group_of_words): #makes a list of lists containing sentences for each single sample 
#   lst = []
#   for i in range(0,840):
#     appendlist = []
#     string = group_of_words[i]
#     appendstr = ' '.join(string)
#     appendlist.append(appendstr)
#     lst.append(appendlist)
#   return lst

# concat_word_groups = Sentences(word_groups)

In [ ]:
# pd.set_option('display.max_colwidth', None)
# MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
# BATCH_SIZE = 16
# N_EPOCHS = 10
# tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
# encoding = tokenizer(concat_word_groups, is_split_into_words = True, max_length = 10, padding = True, truncation="only_first", return_tensors = "tf")
# model_right = TFDistilBertModel.from_pretrained("distilbert-base-uncased")
# output = model_right(encoding)
# last_hidden_states = output.last_hidden_state
# berttensor = tf.convert_to_tensor(last_hidden_states)
# berttensor1 = tf.reshape(last_hidden_states, [840, 7680])

#model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
# optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
# losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer=optimizerr,loss=losss,metrics=['accuracy'])
# model.fit(train_dataset.shuffle(len(concat_word_groups)).batch(BATCH_SIZE),epochs=N_EPOCHS,batch_size=BATCH_SIZE)
# output = model_right(encoding)

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((dict(encoding), Y_data))
# model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)
# optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
# losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# model.compile(optimizer=optimizerr,loss=losss,metrics=['accuracy'])
# model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
#           epochs=N_EPOCHS,
#           batch_size=BATCH_SIZE)


# Neural Network Architecture

## Left Branch : CNN 




In [ ]:
# model = Sequential()
# model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(56,1)))
# model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Flatten())
# model.add(Dense(100, activation='relu'))
# model.add(Dense(50, activation='relu'))
# model.add(Dense(25, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(X_crosstrain,Y_crosstrain, batch_size=50, epochs=150)
# accuracy = model.evaluate(X_crosstest, Y_crosstest, batch_size=50)
# print(accuracy)

In [ ]:
# Y_tensor =tf.convert_to_tensor(Y_tr)

In [ ]:
# height, width, depth = X_tr.shape[0], X_tr.shape[1], X_tr.shape[2]
# input_shape=(width,depth)
# input = Input(shape=input_shape)
# X = Convolution1D(filters=32, kernel_size=3, activation='relu')(input)
# X = Convolution1D(filters=64, kernel_size=3, activation='relu')(X)
# X = Convolution1D(filters=64, kernel_size=3, activation='relu')(X)
# X = Flatten()(X)
# concatenate = tf.keras.layers.Concatenate(axis=1)([X, berttensor1])
# X = Dense(100, activation='relu')(concatenate)
# X = Dense(50, activation='relu')(X)
# X = Dense(10, activation = 'relu')(X)
# output = Dense(1, activation = "sigmoid")(X)

# model = Model(inputs=input, outputs=output)

In [ ]:
# newinput = Input(shape=[10880])
# hidden1 = Dense(20, activation='relu')(newinput)
# hidden2 = Dense(10, activation='relu')(hidden1)
# output = Dense(1, activation='sigmoid')(hidden2)
# model = Model(inputs=newinput, outputs=output)

In [ ]:
# epochs, batch_size, test_batch_size = 50, 50, 50
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.fit(X_tr,Y_tr, batch_size=batch_size, epochs=epochs)

In [ ]:
# def evaluate_model(trainX, trainy, testX, testy): 
# 	epochs, batch_size, test_batch_size = 120, 50, 50
# 	height, width, depth = trainX.shape[0], trainX.shape[1], trainX.shape[2]
# 	model = Sequential()
# 	model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(width,depth)))
# 	model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# 	model.add(Flatten())
# 	model.add(Dense(1, activation='sigmoid'))
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size)
# 	_, accuracy = model.evaluate(testX, testy, batch_size=test_batch_size)
# 	return accuracy

# model_accuracy = evaluate_model(X_crosstrain, Y_crosstrain, X_crosstest, Y_crosstest)